In [1]:
from torch_geometric.datasets.word_net import WordNet18RR

c:\Users\simon\anaconda3\envs\geometric\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = WordNet18RR(root="/tmp/WordNetRR/")

Processing...
Done!


In [5]:
data= dataset[0]

from torch_geometric.utils.convert import to_networkx
import networkx as nx

graph = to_networkx(data)


In [18]:
mask = data.train_mask
indexes = [i for i, x in enumerate(mask) if x]
subgraph = nx.Graph(nx.subgraph(graph, indexes))

In [45]:
components = nx.connected_components(subgraph)
component = list(next(components))
component_graph = nx.subgraph(subgraph, component)

In [48]:
import numpy as np
X = np.random.choice(component, 10).tolist()

In [40]:
# node2comp = {} # TODO: fill
# for component in components:
#     for node in component:
#         node2comp[node] = component

In [51]:
# Y = []
# for x in X:
#     # component = node2comp[x]
#     y = np.random.choice(component, 10).tolist()
#     Y.append(y)

In [54]:
Y = np.random.choice(component, 10).tolist()

In [57]:
[len(nx.shortest_path(subgraph, x, y)) for x, y in zip(X, Y)]

[7, 9, 7, 9, 9, 8, 9, 12, 5, 9]

In [58]:
def get_mode_index(data, mode):
    if mode == "train":
        mask = data.train_mask
    elif mode == "val":
        mask = data.val_mask
    elif mode == "test":
        mask = data.test_mask
    else:
        raise ValueError('Unknown mode. Please use mode in [train, val, test].')
    return [i for i, x in enumerate(mask) if x]

In [78]:
train_indexes = get_mode_index(data, "train")
val_indexes = get_mode_index(data, "val")
test_indexes = get_mode_index(data, "test")

graph = nx.Graph(to_networkx(data)) 
components = nx.connected_components(graph)
node2pool = {}
for component in components:
    componentxtrain = component.intersection(train_indexes) 
    componentxval = component.intersection(val_indexes) 
    componentxtest = component.intersection(test_indexes) 
    # Link node to a component x mask (only nodes of the same component with the same mask should be taken).
    for node in component:
        if node in train_indexes:
            node2pool[node] = list(componentxtrain)
        elif node in val_indexes:
            node2pool[node] = list(componentxval)
        elif node in test_indexes:
            node2pool[node] = list(componentxtest)
        else:
            raise KeyError("?")

pairs = {}
for mode, indexes in zip(['train', 'val', 'test'], [train_indexes, val_indexes, test_indexes]):            
    X = np.random.choice(indexes, 100).tolist()
    Y = []
    for x in X:
        component = node2pool[x] # Select the same component as X to guarantee there exists a path.
        y = int(np.random.choice(component))
        Y.append(y)
    pairs[mode] = {
        "x": X,
        "y": Y,
        "distance": [nx.shortest_path(graph, x, y) for x, y in zip(X, Y)]
    }

KeyError: 59849

In [91]:
data

Data(edge_index=[2, 93003], edge_type=[93003], train_mask=[93003], val_mask=[93003], test_mask=[93003], num_nodes=40943)

In [86]:
components = nx.connected_components(graph)
for comp in components:
    print(91850 in comp)

False
False
False
False
False
False
False
False
False
False
False
False
False
